# 日期型变量的特征提取

在现实中很多观测现象具有周期性，例如太阳黑子黑子有11年的变化周期，一些脉冲星的周期是毫秒量级。在数据分析中，对于日期时间型变量，需要提取其中特征，以便找到与时间相关的规律变化。常用的特征提取有如下：
- 使用时间戳对象的属性来提取日期或时间的特征
- 进行时间处理来提取相对时间特征。

本节介绍从日期型变量进行特征提取的常用方法。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 股票交易数据集

这里使用国内A股一个股票的交易数据，数据格式为csv文件，包括如下字段：
- `date`，交易
- `open`，当日开盘价
- `close`，当日收盘价
- `high`，当日最高价
- `low`，当日开盘价
- `volumn`，成交量
- `code`, 股票代码

首先随机选择一个股票，然后导入该股票的数据文件：

In [2]:
import os
import random

stockcodes = ['300137', '300059', '601288', '600019', '000651'] 
stockcode = random.choice(stockcodes)
csvfile = os.path.join('..', 'data', 'stocks', '{}_lday.csv'.format(stockcode))
stockcode, csvfile

('600019', '..\\data\\stocks\\600019_lday.csv')

In [3]:
stockdata = pd.read_csv(csvfile, parse_dates=['date'])
stockdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 7 columns):
date      657 non-null datetime64[ns]
open      657 non-null float64
close     657 non-null float64
high      657 non-null float64
low       657 non-null float64
volume    657 non-null float64
code      657 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 36.0 KB


## 提取时间序列数据信息

股票数据集`date`列为 Pandas 时间戳数据，可以通过属性`dt`来提取股票交易的年、月、周、时、分、秒等信息。例如有人感觉股票在2月，5月经常上涨；最近在周四常会下跌，可以从时间中提取月与周日的信息。使用如下语句提取：

In [4]:
df = stockdata.copy()
df['month'] = df.date.dt.month
df['weekday'] = df.date.dt.weekday
df.sample(10)

,date,open,close,high,low,volume,code,month,weekday
185,2017-01-23,6.380,6.261,6.389,6.242,821663.0,600019,1,0
454,2018-03-30,8.226,8.103,8.255,7.988,805084.0,600019,3,4
183,2017-01-19,6.491,6.435,6.629,6.417,796856.0,600019,1,3
225,2017-04-25,5.579,5.607,5.625,5.552,332352.0,600019,4,1
528,2018-07-19,7.490,7.730,7.740,7.470,1283208.0,600019,7,3
471,2018-04-26,8.778,8.740,8.816,8.588,525272.0,600019,4,3
247,2017-05-26,5.911,5.957,6.040,5.902,746984.0,600019,5,4
416,2018-01-30,9.520,9.291,9.672,9.253,1781167.0,600019,1,1
462,2018-04-13,8.359,8.321,8.378,8.226,483768.0,600019,4,4
620,2018-12-04,6.860,7.000,7.050,6.830,688046.0,600019,12,1


下面列出一些常用属性及说明：

| 属性名称 | 描述   | 属性名称 | 描述   |
|:---------|:---------|:---------|:---------|
| year    | 年     | weekofyear | 一年中第几周 |
| quarter  | 季     | dayofyear  | 年积日 |
| month   | 月     | dayofweek   | 一周第几天 |
| day    | 日     |  dayofweek   | 星期 |
| hour    | 时     | weekday_name  | 星期名称 |
| minute  | 分     | is_month_start | 是否月初 |
| second  | 秒     | is_month_end | 是否月末 |

## 计算相对时间特征

如果有多个时间戳变量，可以通过计算差值来计算相对时间特征。这里添加一列，表示信号出现时间：

In [5]:
df['signal_date'] = df['date']
df.signal_date.where(df.index % 10 == 0, np.nan, inplace=True)
df['signal_date'].fillna(method='ffill', inplace=True)
df.head()

,date,open,close,high,low,volume,code,month,weekday,signal_date
0,2016-01-04,5.096,4.769,5.115,4.742,447328.0,600019,1,0,2016-01-04
1,2016-01-05,4.732,4.942,5.033,4.687,867079.0,600019,1,1,2016-01-04
2,2016-01-06,4.951,5.433,5.433,4.905,2919012.0,600019,1,2,2016-01-04
3,2016-01-07,5.324,4.896,5.333,4.887,525823.0,600019,1,3,2016-01-04
4,2016-01-08,5.096,5.388,5.388,5.015,2243776.0,600019,1,4,2016-01-04


下面语句用来提取信号发生持续时间：

In [6]:
# 信号发生的天数
df['occur_days'] = (df['date'] - df['signal_date']).dt.days
df.head()

,date,open,close,high,low,volume,code,month,weekday,signal_date,occur_days
0,2016-01-04,5.096,4.769,5.115,4.742,447328.0,600019,1,0,2016-01-04,0
1,2016-01-05,4.732,4.942,5.033,4.687,867079.0,600019,1,1,2016-01-04,1
2,2016-01-06,4.951,5.433,5.433,4.905,2919012.0,600019,1,2,2016-01-04,2
3,2016-01-07,5.324,4.896,5.333,4.887,525823.0,600019,1,3,2016-01-04,3
4,2016-01-08,5.096,5.388,5.388,5.015,2243776.0,600019,1,4,2016-01-04,4
